In [1]:
import gym
import tensorflow as tf
from collections import deque
import numpy as np
from random import sample, randint
from tensorflow import keras
from Helper import argmax, softmax
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical

C:\Users\nikma\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
C:\Users\nikma\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:637: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
C:\Users\nikma\anaconda3\lib\site-packages\tensorflow\python\framework\tensor_util.py:176: DeprecationWarning: `np.object` is a de

In [2]:
def get_model(ip_shape,lr,op_shape,summary = True):
    '''
    get_model(ip_shape,lr,op_shape,summary = True):
    creates and returns a model and prints it's summary based on summary flag
    '''
    
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=ip_shape))
    model.add(keras.layers.Dense(24, activation="relu"))
    model.add(keras.layers.Dense(24, activation="relu"))
    model.add(keras.layers.Dense(24, activation="relu"))
    model.add(keras.layers.Dense(op_shape, activation="linear"))

    # compile model
    model.compile(loss="mean_squared_error",optimizer=keras.optimizers.Adam(learning_rate=lr),metrics=["accuracy"])
    if summary == True:
        print(model.summary())
    return(model)

C:\Users\nikma\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
class experience_deque:
    '''
    __init__(self, max_len):
    
    # methods:
    add_experience(self,s,a,r,s_next,done):
    get_batch(self, batch_size):
    '''
    def __init__(self, max_len):
        '''initialisation'''
        #initialise max buffer length
        self.deque_size = max_len
        
        # initialise buffer for live deque length
        self.live_ds = 0
        
        # initialise experience buffers
        self.s_experience = deque(maxlen = self.deque_size)
        self.s_next_experience = deque(maxlen = self.deque_size)
        self.a_experience = deque(maxlen = self.deque_size)
        self.r_experience = deque(maxlen = self.deque_size)
        self.d_experience = deque(maxlen = self.deque_size)
        
        
        
        
    def add_experience(self,s,a,r,s_next,done):
        '''
        add_experience(self,s,a,r,s_next,done)
        add an experience to the deques
        '''
        self.s_experience.append(s)
        self.s_next_experience.append(s_next)
        self.a_experience.append(a)
        self.r_experience.append(r)
        self.d_experience.append(done)
        
        #update live deque size
        self.live_ds = len(self.s_experience)
        
        

        
        
    def get_batch(self, batch_size):
        '''
        get_batch(self, batch_size):
        generate random samples from experiences
        returns them
        '''
        # warn that deque is not full
        if self.live_ds < self.deque_size:
            if self.live_ds%1000 == 0:
                print(self.live_ds)
                
#             print("deque is not full, current size is : ", self.live_ds)
#             if batch_size > self.live_ds:
#                 print("batch size bigger than live deque size (bs,lds): ", batch_size, self.live_ds)
#             else:
#                 print("sampling from incomplete deque (bs,lds): ", batch_size, self.live_ds)
        
        # get random indices
        ind = sample(range(self.live_ds), batch_size)
        
        # sample from all deques
        s_sampled = np.asarray(self.s_experience)[ind]
        s_next_sampled = np.asarray(self.s_next_experience)[ind]
        a_sampled = np.asarray(self.a_experience)[ind]
        r_sampled = np.asarray(self.r_experience)[ind]
        d_sampled = np.asarray(self.d_experience)[ind]
        
        return (s_sampled,s_next_sampled,a_sampled,r_sampled,d_sampled)
        

In [4]:
class DQNagent:
    '''
    __init__(self, n_states, n_actions, learning_rate, gamma, max_len, batch_size,
    er = True, tn = True, conv = False, summary = True, verbose = 0):
    Sets up a model and provides handy methods to interact with it
    '''
    
    def __init__(self, env, buffer, live_model, target_model,  gamma, batch_size, er = True,
                 tn = True, summary = True, verbose = 2):
        '''Iniitialization function for class DQNagent, read the __docs__'''
        
        # used for 
        self.buffer = buffer
        self.live_model = live_model
        self.target_model = target_model
        self.n_states = env.observation_space.shape
        self.n_actions = env.action_space.n
        self.gamma = gamma
#         self.deque_size = max_len
        self.batch_size = batch_size
        self.verbose = verbose


        
        
    def select_action(self, state, policy='egreedy', epsilon=None, temp=None):
        '''
        select_action(self, s, policy='egreedy', epsilon=None, temp=None):
        selects action based on policy specified
        returns action
        '''
        state = state.reshape(1,4)
        if policy == 'egreedy':
            if epsilon is None:
                raise KeyError("Provide an epsilon")
                
            # TO DO: Add own code
            exploit = np.random.choice([0,1],p = [epsilon,1-epsilon])
            if exploit:
#                 print(q[0])
                a = argmax((self.live_model.predict(state))[0])
#                 print("exploiting: ",a)
            else:
                a = np.random.randint(0,self.n_actions) # Replace this with correct action selection
#                 print("exploring: ",a)
                
#         elif policy == 'softmax':
#             if temp is None:
#                 raise KeyError("Provide a temperature")
                
#             # TO DO: Add own code
#             a = np.random.randint(0,self.n_actions) # Replace this with correct action selection
#             print("action selected :", a)
        return a
                

    '''
    def update(self,s,a,r,s_next,done):
        #perform a Q-learning update

        # TO DO: Add own code
        G = r + ( self.gamma * np.max(self.Q_sa[s_next,:]) )
        self.Q_sa[s,a] = self.Q_sa[s,a] + ( self.learning_rate * ( G - self.Q_sa[s,a] ) )
        pass
    
#                     def fit_batch(env, model, target_model, batch):
#                        observations, actions, rewards, next_observations, dones = batch
#                        # Predict the Q values of the next states. Passing ones as the action mask.
#                        next_q_values = predict(env, target_model, next_observations)
#                        # The Q values of terminal states is 0 by definition.
#                        next_q_values[dones] = 0.0
#                        # The Q values of each start state is the reward + gamma * the max next state Q value
#                        q_values = rewards + DISCOUNT_FACTOR_GAMMA * np.max(next_q_values, axis=1)
#                        one_hot_actions = np.array([one_hot_encode(env.action_space.n, action) for action in actions])
#                        history = model.fit(
#                            x=[observations, one_hot_actions],
#                            y=one_hot_actions * q_values[:, None],
#                            batch_size=BATCH_SIZE,
#                            verbose=0,
#                        )
#                        return history.history['loss'][0]

    '''
    
    def update_er_tn(self,batch_of_ss, batch_of_as, batch_of_sns, batch_of_rs,batch_of_ds):
        '''
        update_er_tn(self,batch):
        perform a Q-learning update
        '''

        # get target Q values for the batch
        q_next_batch = self.target_model.predict(batch_of_sns)
        
        # calculate targets and assign done rewards
        G1 = self.live_model.predict(batch_of_ss)
        cat_boa = to_categorical(batch_of_as,num_classes = 2)
        cat_inv_boa = to_categorical(np.invert(batch_of_as),num_classes = 2)
        G1 = G1 * cat_inv_boa
        G2 = batch_of_rs + ( self.gamma * np.max(q_next_batch, axis = 1) )
        G2 = np.where(batch_of_ds == True, batch_of_rs, G2)
        G2 = G2.reshape(G2.shape[0],1)
        G2 = G2 * cat_boa
        G_batch = G1+G2

        
        # update live_network
        history = self.live_model.fit(batch_of_ss,G_batch, batch_size = self.batch_size, verbose = 0)
        
        return history
    
    
    
    
    def synch_weights(self):
        '''
        synch_weights(self):
        synchronises target model weights
        '''
        self.target_model.set_weights(self.live_model.get_weights())
        print("weights synched")

In [5]:
def decay_eps(epsilon = 1, min_epsilon = 0.01, decay_rate = 0.95):
    '''
    decay_eps(max_epsilon = 1, min_epsilon = 0.01, decay_rate = 0.95):
    decay the epsilon value
    '''
    epsilon *= decay_rate
    return max(epsilon, min_epsilon)

In [6]:
# main loop
def Qlearn(learning_rate, epsilon, buffer_size, n_eps, max_timesteps,batch_size, min_buff = 100, decay_epsilon = True):
    
    # initialise environment
    env = gym.make('CartPole-v1')
    
    # create buffers for kpi
    reward_per_step_per_ep = [] # list of cum_reward at each step [n_eps, n_timesteps]
    cum_reward_per_ep = [] # list of final rewards [n_eps]

    # initialise networks
    live_net = get_model(env.observation_space.shape,learning_rate,env.action_space.n)
    target_net = get_model(env.observation_space.shape,learning_rate,env.action_space.n)
    
    # initialise buffers
    buffer = experience_deque(max_len = buffer_size)
    
    # initialise agent
#         def __init__(self, env, buffer, live_model, target_model,  gamma, batch_size, er = True,
#                  tn = True, summary = True, verbose = 2):
    agent = DQNagent(env, buffer, live_net, target_net, gamma, batch_size)
    
    # count for target net update
    step_count = 0
    
    # loop over eps
    for ep_num in range(n_eps):
        s = env.reset()
        rewards = []
        cum_reward = 0
        # loop over timesteps
        for step in range(max_timesteps):
            step_count += 1
#             env.render()
            
            # select action
            a = agent.select_action(state = s,policy = 'egreedy', epsilon = epsilon)
#             print(a)
#             print(epsilon)

            
            # play a step
            s_next,reward,done,_ = env.step(a)
            cum_reward += reward
            
            # save experience
            buffer.add_experience(s,a,reward,s_next,done)
        
            if buffer.live_ds >= min_buff:
                batch_size = min(int(0.7*buffer.live_ds),500)
                
                # run a DQN training loop
                s_exp_batch,s_next_exp_batch,a_exp_batch,r_exp_batch,d_exp_batch = buffer.get_batch(batch_size)
                history = agent.update_er_tn(s_exp_batch,a_exp_batch,s_next_exp_batch,r_exp_batch,d_exp_batch)
#                 print("loss = ",history.history["loss"][0])

                # copy weights to target_network
                if step_count%synch_weights_freq == 0:
                    agent.synch_weights()
    #             print("trainstep success")
    
#             else:
#                 print("only adding to buffer")

            
            # create reward per step
            rewards.append(cum_reward)
            
            # check if done
            if done:
#                 print("done = ",done)
                print("ep = ", ep_num, "    steps = ", step,"ep reward = ",cum_reward)
                break
                
        
            # set new state
            s = s_next
        
        #
        reward_per_step_per_ep.append(rewards)
        cum_reward_per_ep.append(cum_reward)
        # Decay probability of taking random action
        if decay_epsilon:
            epsilon = decay_eps(epsilon,eps_min)
                
    return (reward_per_step_per_ep,cum_reward_per_ep)
    

In [11]:
n_eps = 1000
buffer_size = 10000
synch_weights_freq = 20
learning_rate = 0.01
gamma = 0.995
epsilon = 1
batch_size = 24
min_buff = 1000
max_timesteps = 500
eps_min = 0.01

In [12]:
reward_per_step_per_ep,cum_reward_per_e = Qlearn(learning_rate, epsilon, buffer_size, n_eps, max_timesteps, batch_size, min_buff = min_buff)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_16 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_17 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_18 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 50        
Total params: 1,370
Trainable params: 1,370
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_5"
_________________________________________________________________
Layer (type)             

weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  77     steps =  375 ep reward =  376.0
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  78     steps =  125 ep reward =  126.0
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
5000
weights synched
ep =  79     steps =  205 ep reward =  206.0
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  80     steps =  499 ep reward =  500.0
weights synched

weights synched
weights synched
weights synched
weights synched
ep =  117     steps =  171 ep reward =  172.0
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  118     steps =  112 ep reward =  113.0
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  119     steps =  86 ep reward =  87.0
weights synched
weights synched
weights synched
weights synched
ep =  120     steps =  94 ep reward =  95.0
weights synched
weights synched
ep =  121     steps =  36 ep reward =  37.0
weights synched
weights synched
weights synched
weights synched
ep =  122     steps =  73 ep reward =  74.0
weights synched
weights synched
weights synched
weights synched
ep =  123     steps =  79 ep reward =  80.0
weights synched
weights synched
weights synched
ep =  124     steps =  71 ep reward =  72.0
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  125     steps =  107 ep reward =  108.0
weight

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x0000021A42333310>
Traceback (most recent call last):
  File "C:\Users\nikma\anaconda3\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 545, in __del__
    gen_dataset_ops.delete_iterator(
  File "C:\Users\nikma\anaconda3\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1262, in delete_iterator
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt: 


weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  146     steps =  499 ep reward =  500.0
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  147     steps =  258 ep reward =  259.0
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  148     steps =  147 ep reward =  148.0
weights synched
weights synched
weights synched
weights synched
weights synched
weights synched
ep =  149     steps =  124 ep reward =  125.0
weights synched
weights synched
weights synched
weights synched
weights synched


KeyboardInterrupt: 